# SKU Discount Handler Module

This module handles the complete lifecycle of SKU discounts based on Module 3 recommendations.

**Note:** This module is called from Module 3. Market data should already be refreshed in Module 3 before calling this handler.

## Workflow
1. **Deactivate Active SKU Discounts** - Clear existing discounts before creating new ones
2. **Select Target Retailers** - Identify which retailers should receive the discount
3. **Calculate Target Price** - Find optimal discounted price based on market/margin data
4. **Calculate Discount Percentage** - Derive discount from current price vs target price
5. **Structure DataFrame** - Prepare data in API-expected format
6. **Push SKU Discount** - Upload to MaxAB API

## Price Selection Logic (from HH-V3)
- For **zero demand** or **overstock** (doh > 45): Aggressive price reduction
- For **normal products**: Find best price from market tiers and margin tiers
- **UTH Status adjustments**:
  - On Track: Keep same discount if previously discounted
  - Dropping: More aggressive discount (lower price)
  - Growing: Less aggressive discount

## Input Requirements

**From Module 3 (with market data already refreshed):**
- `product_id`, `warehouse_id`, `sku`, `cohort_id`, `brand`, `cat`
- `activate_sku_discount` - Boolean flag
- `current_price`, `wac_p`, `doh`, `uth_qty` (zero demand = uth_qty == 0)
- `uth_status` - UTH performance status
- `active_sku_disc_pct` - Current active SKU discount percentage (if any)
- `target_margin`, `min_boundary` - For fallback price calculations

*Market Margins (converted to prices using `wac / (1 - margin)`):*
- `below_market`, `market_min`, `market_25`, `market_50`, `market_75`, `market_max`, `above_market`

*Margin Tiers (converted to prices using `wac / (1 - margin)`):*
- `margin_tier_below`, `margin_tier_1`, `margin_tier_2`, `margin_tier_3`, `margin_tier_4`, `margin_tier_5`, `margin_tier_above_1`, `margin_tier_above_2`

## Output
- Deactivated existing SKU discounts
- Created new SKU discounts for qualifying SKUs
- Summary report

## Usage (called from Module 3)
```python
%run sku_discount_handler.ipynb
result = process_sku_discounts(df_output, mode='testing')
```


In [ ]:
# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import pandas as pd
import numpy as np
import json
import time
import base64
import os
from datetime import datetime, timedelta
import pytz

# AWS for secrets management
import boto3
from botocore.exceptions import ClientError

# HTTP requests for API calls
import requests

# Progress bar
from tqdm import tqdm

# Import setup_environment_2 for credentials
import sys
sys.path.append('..')
import setup_environment_2

# Initialize environment (loads Snowflake credentials)
setup_environment_2.initialize_env()

# Note: Market data is passed from Module 3 (already refreshed)
# No need to import market_data_module here

# Cairo Timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
TODAY = CAIRO_NOW.date()

# =============================================================================
# CONFIGURATION
# =============================================================================
# Default discount settings
DEFAULT_DISCOUNT_DURATION_HOURS = 12  # Discount valid until next run

# =============================================================================
# EXCLUSION LISTS
# =============================================================================
# Categories to exclude from SKU discounts
EXCLUDED_CATEGORIES = [
    'كروت شحن',  # Recharge cards
]

# Brands to exclude from SKU discounts
EXCLUDED_BRANDS = [
    'فيوري',
    'العروسة',
]

# =============================================================================
# PRICE SELECTION PARAMETERS
# =============================================================================
# Days on Hand threshold for overstock
DOH_OVERSTOCK_THRESHOLD = 45

# Max discount percentage (cap)
MAX_DISCOUNT_PERCENT = 5.0

# Minimum discount percentage (skip if below)
MIN_DISCOUNT_PERCENT = 0.25

# Maximum price reduction from current (as decimal, e.g., 0.05 = 5%)
MAX_PRICE_REDUCTION_RATIO = 0.05

# Minimum price reduction from current (as decimal)
MIN_PRICE_REDUCTION_RATIO = 0.0025

# =============================================================================
# COLUMN DEFINITIONS
# =============================================================================
# Market margin columns (prices derived from these using wac / (1 - margin))
MARKET_MARGIN_COLS = [
    'below_market', 'market_min', 'market_25', 'market_50', 
    'market_75', 'market_max', 'above_market'
]

# Margin tier columns (prices derived from these using wac / (1 - margin))
MARGIN_TIER_COLS = [
    'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
    'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2'
]

print(f"SKU Discount Handler loaded at {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')} Cairo time")
print(f"Excluded categories: {EXCLUDED_CATEGORIES}")
print(f"Excluded brands: {EXCLUDED_BRANDS}")


In [ ]:
# =============================================================================
# AWS & API FUNCTIONS
# =============================================================================

def get_secret(secret_name: str) -> str:
    """
    Retrieve a secret from AWS Secrets Manager.
    
    Args:
        secret_name: The name/path of the secret in AWS Secrets Manager
    
    Returns:
        Secret string (JSON format) or decoded binary
    """
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name=region_name)

    try:
        response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        print(f"AWS Error: {e}")
        raise e
    
    if 'SecretString' in response:
        return response['SecretString']
    return base64.b64decode(response['SecretBinary'])


def get_access_token(url: str, client_id: str, client_secret: str) -> str:
    """
    Get OAuth2 access token for MaxAB API authentication.
    """
    response = requests.post(
        url,
        data={
            "grant_type": "password",
            "username": API_USERNAME,
            "password": API_PASSWORD
        },
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def _get_api_token() -> str:
    """
    Get a fresh API token for MaxAB API requests.
    """
    return get_access_token(
        'https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
        'main-system-externals',
        API_SECRET
    )

print("AWS & API functions defined ✓")


In [ ]:
# =============================================================================
# API CREDENTIALS INITIALIZATION
# =============================================================================
# Load API credentials from AWS Secrets Manager

pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
API_USERNAME = pricing_api_secret["egypt_username"]
API_PASSWORD = pricing_api_secret["egypt_password"]
API_SECRET = pricing_api_secret["egypt_secret"]

print("✓ API credentials loaded successfully")


In [ ]:
# =============================================================================
# FUNCTION 1: DEACTIVATE ACTIVE SKU DISCOUNTS
# =============================================================================
# Reference: Mustafa/Deactivate_HH.ipynb

import snowflake.connector
import math

def get_snowflake_timezone() -> str:
    """Get the Snowflake server timezone."""
    con = snowflake.connector.connect(
        user     = os.environ["SNOWFLAKE_USERNAME"],
        account  = os.environ["SNOWFLAKE_ACCOUNT"],
        password = os.environ["SNOWFLAKE_PASSWORD"],
        database = os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute("SHOW PARAMETERS LIKE 'TIMEZONE'")
        result = cur.fetchone()
        return result[1] if result else 'UTC'
    finally:
        cur.close()
        con.close()

# Get Snowflake timezone for query conversion
TIMEZONE = get_snowflake_timezone()
print(f"Snowflake timezone: {TIMEZONE}")

def query_snowflake(query: str) -> pd.DataFrame:
    """
    Execute a query against Snowflake and return results as DataFrame.
    """
    con = snowflake.connector.connect(
        user     = os.environ["SNOWFLAKE_USERNAME"],
        account  = os.environ["SNOWFLAKE_ACCOUNT"],
        password = os.environ["SNOWFLAKE_PASSWORD"],
        database = os.environ["SNOWFLAKE_DATABASE"]
    )

    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        
        column_names = [col[0] for col in cur.description]
        results = cur.fetchall()
        
        if not results:
            out = pd.DataFrame(columns=[name.lower() for name in column_names])
        else:
            out = pd.DataFrame(np.array(results), columns=column_names)
            out.columns = out.columns.str.lower()
        
        return out
        
    except Exception as e:
        print(f"❌ Query error: {e}")
        raise
        
    finally:
        cur.close()
        con.close()


def _call_deactivate_api(ids: list) -> requests.Response:
    """
    Call MaxAB API to deactivate SKU discounts by their IDs.
    
    Args:
        ids: List of SKU discount IDs to deactivate
        
    Returns:
        API response object
    """
    token = _get_api_token()
    
    url = "https://api.maxab.info/commerce/api/admins/v1/sku-discounts/status"
    
    headers = {
        'Authorization': f'bearer {token}',
        'Content-Type': 'application/json'
    }
    
    payload = {
        "ids": ids,
        "isActivate": False
    }
    
    response = requests.put(url, headers=headers, json=payload)
    return response


def get_active_sku_discount_ids() -> list:
    """
    Query Snowflake to get all currently active SKU discount IDs.
    
    Returns:
        List of active SKU discount IDs
    """
    query = f'''
    SELECT sd.id AS sku_discount_id
    FROM SKU_DISCOUNTS sd
    WHERE active = 'true'
    AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) BETWEEN start_at AND end_at
    AND name_en = 'Special Discounts'
    '''
    
    df = query_snowflake(query)
    return df['sku_discount_id'].unique().tolist()


def deactivate_active_sku_discounts(mode: str = 'testing') -> dict:
    """
    Deactivate all currently active SKU discounts (name = 'Special Discounts').
    
    This function should be called FIRST before creating new discounts
    to ensure a clean slate (no duplicate/overlapping discounts).
    
    Process:
    1. Query Snowflake to get all active SKU discount IDs
    2. Break IDs into chunks of 10
    3. Call API for each chunk to deactivate
    
    Args:
        mode: 'testing' (don't call API) or 'live' (call API)
    
    Returns:
        dict with:
            - success: bool
            - deactivated_count: int
            - failed_count: int
            - errors: list
    """
    print(f"{'🧪' if mode == 'testing' else '🚀'} Deactivating SKU discounts (mode: {mode})")
    
    result = {
        "success": True,
        "deactivated_count": 0,
        "failed_count": 0,
        "errors": []
    }
    
    # Step 1: Get active SKU discount IDs from Snowflake
    print("  Querying active SKU discounts from Snowflake...")
    ids_list = get_active_sku_discount_ids()
    print(f"  Found {len(ids_list)} active SKU discounts to deactivate")
    
    if len(ids_list) == 0:
        print("  No active SKU discounts to deactivate")
        return result
    
    if mode == 'testing':
        print(f"  🧪 [TESTING] Would deactivate {len(ids_list)} SKU discounts (skipped)")
        result['deactivated_count'] = len(ids_list)
        return result
    
    # Step 2: Deactivate in chunks of 10
    chunk_size = 10
    total_chunks = math.ceil(len(ids_list) / chunk_size)
    failed_chunks = []
    
    print(f"  Deactivating in {total_chunks} chunks...")
    
    for i in tqdm(range(0, len(ids_list), chunk_size), total=total_chunks, desc="Deactivating SKU Discounts"):
        chunk = ids_list[i:i + chunk_size]
        
        response = _call_deactivate_api(chunk)
        
        if response.status_code == 200:
            result['deactivated_count'] += len(chunk)
        else:
            result['failed_count'] += len(chunk)
            failed_chunks.append({"ids": chunk, "error": response.text})
    
    # Step 3: Report results
    print(f"\n  ✓ Completed! Deactivated: {result['deactivated_count']}, Failed: {result['failed_count']}")
    
    if failed_chunks:
        result['success'] = False
        result['errors'] = failed_chunks
        print(f"  ✗ Failed chunks: {len(failed_chunks)}")
        for fail in failed_chunks[:5]:  # Show first 5 failures
            print(f"    - IDs: {fail['ids']}, Error: {fail['error'][:100]}...")
    
    return result


print("Function 1: deactivate_active_sku_discounts() defined ✓")


In [ ]:
# =============================================================================
# FUNCTION 2: SELECT TARGET RETAILERS
# =============================================================================
# Reference: HH-V3.ipynb retailer selection logic

# Import retailer selection queries from queries_module
%run queries_module.ipynb


def create_sku_tuple_string(df: pd.DataFrame) -> str:
    """
    Create a tuple string from DataFrame for SQL VALUES clause.
    Format: "(product_id, warehouse_id), (product_id, warehouse_id), ..."
    
    Args:
        df: DataFrame with product_id and warehouse_id columns
        
    Returns:
        String of tuples for SQL VALUES clause
    """
    df = df.copy()
    df['tuple'] = df[['product_id', 'warehouse_id']].apply(tuple, axis=1)
    tuple_list = list(df['tuple'].unique())
    tuple_str = str(tuple_list)[1:-1]  # Remove outer brackets
    return tuple_str


def select_target_retailers(df_skus: pd.DataFrame) -> pd.DataFrame:
    """
    Select which retailers should receive the SKU discount.
    
    This function implements the HH-V3 retailer selection logic:
    1. Filter SKUs that got valid discounts (discount > 0)
    2. Query 4 types of target retailers:
       - Churned/dropped: Were buying but dropped >30%
       - Category buyers: Buy the category but not this product
       - Out of cycle: Should have reordered by now
       - View no orders: Viewed brand but didn't order
    3. Apply exclusions (failed orders, inactive, wholesale, existing SKU discounts)
    4. Remove retailers who already have quantity discounts on the product
    5. Match retailers to their main warehouse
    
    Args:
        df_skus: DataFrame with SKUs to discount (must have discount_percentage column)
    
    Returns:
        DataFrame with retailer_id column added (one row per retailer-product combination)
    """
    print("\n  Selecting target retailers...")
    
    # =========================================================================
    # Step 1: Filter SKUs with valid discounts
    # =========================================================================
    df_valid = df_skus[df_skus['discount_percentage'] > 0].copy()
    
    if len(df_valid) == 0:
        print("    No SKUs with valid discounts to target retailers for")
        df_result = df_skus.copy()
        df_result['retailer_id'] = None
        return df_result
    
    print(f"    SKUs with valid discounts: {len(df_valid)}")
    
    # =========================================================================
    # Step 2: Create tuple string for queries
    # =========================================================================
    selected_skus_tuple = create_sku_tuple_string(df_valid[['product_id', 'warehouse_id']].drop_duplicates())
    print(f"    Created tuple string for {df_valid[['product_id', 'warehouse_id']].drop_duplicates().shape[0]} unique product-warehouse combinations")
    
    # =========================================================================
    # Step 3: Query all retailer sources
    # =========================================================================
    print("\n    Querying retailer sources...")
    
    # Query 1: Churned/Dropped retailers
    try:
        df_churned_dropped = get_churned_dropped_retailers(selected_skus_tuple)
    except Exception as e:
        print(f"    ⚠ Churned/dropped query failed: {e}")
        df_churned_dropped = pd.DataFrame(columns=['retailer_id', 'product_id', 'warehouse_id'])
    
    # Query 2: Category not product retailers
    try:
        df_cat_not_product = get_category_not_product_retailers(selected_skus_tuple)
    except Exception as e:
        print(f"    ⚠ Category-not-product query failed: {e}")
        df_cat_not_product = pd.DataFrame(columns=['retailer_id', 'product_id', 'warehouse_id'])
    
    # Query 3: Out of cycle retailers
    try:
        df_out_of_cycle = get_out_of_cycle_retailers(selected_skus_tuple)
    except Exception as e:
        print(f"    ⚠ Out-of-cycle query failed: {e}")
        df_out_of_cycle = pd.DataFrame(columns=['retailer_id', 'product_id', 'warehouse_id'])
    
    # Query 4: View no orders retailers
    try:
        df_view_no_orders = get_view_no_orders_retailers(selected_skus_tuple)
    except Exception as e:
        print(f"    ⚠ View-no-orders query failed: {e}")
        df_view_no_orders = pd.DataFrame(columns=['retailer_id', 'product_id', 'warehouse_id'])
    
    # =========================================================================
    # Step 4: Combine all retailer sources
    # =========================================================================
    print("\n    Combining retailer sources...")
    all_retailers = pd.concat([
        df_churned_dropped,
        df_cat_not_product,
        df_out_of_cycle,
        df_view_no_orders
    ]).drop_duplicates().reset_index(drop=True)
    
    print(f"    Total retailer-product combinations before filtering: {len(all_retailers)}")
    
    if len(all_retailers) == 0:
        print("    ⚠ No retailers found from any source")
        df_result = df_skus.copy()
        df_result['retailer_id'] = None
        return df_result
    
    # =========================================================================
    # Step 5: Get retailer main warehouse
    # =========================================================================
    print("\n    Getting retailer main warehouses...")
    try:
        df_ret_wh = get_retailer_main_warehouse()
        all_retailers = all_retailers.merge(
            df_ret_wh, 
            on=['retailer_id', 'warehouse_id'], 
            how='left'
        )
        all_retailers = all_retailers.fillna(0)
        
        # Rank and filter to main warehouse only
        all_retailers['rank'] = all_retailers.groupby(['retailer_id'])['last_wh'].rank(
            method='dense', ascending=False
        ).astype(int)
        all_retailers = all_retailers[all_retailers['rank'] == 1]
        all_retailers = all_retailers.drop(columns=['last_wh', 'rank'])
    except Exception as e:
        print(f"    ⚠ Retailer warehouse query failed: {e}")
    
    print(f"    Retailers after warehouse filter: {len(all_retailers)}")
    
    # =========================================================================
    # Step 6: Apply exclusions
    # =========================================================================
    print("\n    Applying exclusions...")
    try:
        df_excluded = get_excluded_retailers()
        excluded_ids = set(df_excluded['retailer_id'].unique())
        before = len(all_retailers)
        all_retailers = all_retailers[~all_retailers['retailer_id'].isin(excluded_ids)]
        print(f"    Excluded {before - len(all_retailers)} retailers (failed orders, inactive, wholesale, existing discounts)")
    except Exception as e:
        print(f"    ⚠ Exclusion query failed: {e}")
    
    # =========================================================================
    # Step 7: Remove retailers with quantity discounts on same product
    # =========================================================================
    print("\n    Removing retailers with existing quantity discounts...")
    try:
        df_qd = get_retailers_with_quantity_discount()
        if len(df_qd) > 0:
            df_qd['have_quantity'] = 1
            all_retailers = all_retailers.merge(
                df_qd[['retailer_id', 'product_id', 'have_quantity']], 
                on=['retailer_id', 'product_id'], 
                how='left'
            )
            before = len(all_retailers)
            all_retailers = all_retailers[all_retailers['have_quantity'].isna()]
            all_retailers = all_retailers.drop(columns=['have_quantity'])
            print(f"    Removed {before - len(all_retailers)} retailer-product combinations with existing QD")
    except Exception as e:
        print(f"    ⚠ Quantity discount query failed: {e}")
    
    # =========================================================================
    # Step 8: Final cleanup
    # =========================================================================
    all_retailers = all_retailers.drop_duplicates(
        subset=['product_id', 'warehouse_id', 'retailer_id']
    ).reset_index(drop=True)
    
    print(f"\n    ✓ Final retailer-product combinations: {len(all_retailers)}")
    print(f"    ✓ Unique retailers: {all_retailers['retailer_id'].nunique()}")
    print(f"    ✓ Unique products: {all_retailers['product_id'].nunique()}")
    
    # =========================================================================
    # Step 9: Merge with SKU data
    # =========================================================================
    # Convert retailer_id and product_id to numeric for merging
    for col in ['retailer_id', 'product_id', 'warehouse_id']:
        if col in all_retailers.columns:
            all_retailers[col] = pd.to_numeric(all_retailers[col], errors='coerce')
    
    # Merge product data with retailer selections
    df_result = df_valid[['product_id', 'warehouse_id', 'cohort_id', 'sku', 'brand', 'cat',
                          'current_price', 'new_price', 'wac_p', 'target_price', 
                          'discount_percentage', 'discount_source']].merge(
        all_retailers[['retailer_id', 'product_id', 'warehouse_id']], 
        on=['product_id', 'warehouse_id'],
        how='inner'
    )
    
    print(f"\n    ✓ Final output rows: {len(df_result)}")
    
    return df_result


print("Function 2: select_target_retailers() defined ✓")
print("  - Queries 4 retailer sources (churned, category, cycle, view)")
print("  - Applies exclusions (failed orders, inactive, wholesale)")
print("  - Removes retailers with existing quantity discounts")


In [ ]:
# =============================================================================
# FUNCTION 3: PRICE SELECTION & DISCOUNT CALCULATION (Adapted from HH-V3)
# =============================================================================

def margin_to_price(wac: float, margin: float) -> float:
    """
    Convert margin to price.
    Price = WAC / (1 - margin)
    
    Args:
        wac: Weighted Average Cost
        margin: Margin as decimal (e.g., 0.15 for 15%)
    
    Returns:
        Price, or np.nan if invalid
    """
    if pd.isna(margin) or margin >= 1 or wac <= 0:
        return np.nan
    return wac / (1 - margin)


def build_candidate_prices(row: pd.Series) -> list:
    """
    Build a list of candidate prices from market margins and margin tiers.
    
    Converts margins to prices using: price = wac / (1 - margin)
    
    Args:
        row: Series containing price data
    
    Returns:
        Sorted list of unique valid prices
    """
    wac = float(row.get('wac_p', 0) or 0)
    if wac <= 0:
        return []
    
    prices_list = []
    
    # 1. Add prices from market margins
    for col in MARKET_MARGIN_COLS:
        margin = row.get(col)
        if margin and not pd.isna(margin):
            price = margin_to_price(wac, float(margin))
            if not pd.isna(price) and price > 0:
                prices_list.append(price)
    
    # 2. Add prices from margin tiers
    for col in MARGIN_TIER_COLS:
        margin = row.get(col)
        if margin and not pd.isna(margin):
            price = margin_to_price(wac, float(margin))
            if not pd.isna(price) and price > 0:
                prices_list.append(price)
    
    # Clean: remove 0, NaN, inf and duplicates
    cleaned_prices = list({
        x for x in prices_list 
        if x > 0 and not pd.isna(x) and np.isfinite(x)
    })
    
    return sorted(cleaned_prices)


def select_target_price(
    candidate_prices: list,
    current_price: float,
    wac: float,
    target_margin: float,
    min_boundary: float,
    is_zero_demand: bool,
    doh: float,
    uth_status: str,
    old_discount_price: float = None
) -> tuple:
    """
    Select optimal target price from candidates based on HH-V3 logic.
    
    Logic:
    - Zero demand or Overstock: More aggressive - find first valid lower price
    - On Track: Keep old discount if available, else find moderate reduction
    - Dropping: More aggressive discount (go lower than old discount)
    - Growing: Less aggressive (stay closer to current price)
    
    Args:
        candidate_prices: Sorted list of valid prices
        current_price: Current selling price
        wac: Weighted Average Cost
        target_margin: Target margin for the product
        min_boundary: Minimum acceptable margin
        is_zero_demand: True if zero demand product
        doh: Days on Hand
        uth_status: UTH performance status
        old_discount_price: Previous discounted price if any
    
    Returns:
        Tuple of (target_price, source_description)
    """
    if current_price <= 0 or wac <= 0:
        return 0.0, 'invalid_data'
    
    target_price = 0.0
    source = ''
    
    current_margin = (current_price - wac) / current_price
    is_overstock = doh > DOH_OVERSTOCK_THRESHOLD
    
    # Convert to numpy array for vectorized operations
    prices_arr = np.array(candidate_prices)
    
    # Minimum acceptable price (WAC * 0.9 allows small loss margin)
    min_acceptable_price = wac * 0.9
    
    # =========================================================================
    # CASE 1: Zero Demand or Overstock - Aggressive Price Reduction
    # =========================================================================
    if is_zero_demand or is_overstock:
        source = 'zero_demand' if is_zero_demand else 'overstock'
        
        # Find first price that:
        # - Is at or above minimum acceptable (wac * 0.9)
        # - Reduces price by at most 5% from current
        diffs = (prices_arr - current_price) / current_price
        valid_mask = (prices_arr >= min_acceptable_price) & (diffs >= -MAX_PRICE_REDUCTION_RATIO)
        valid_prices = prices_arr[valid_mask]
        
        if len(valid_prices) > 0:
            # Take the lowest valid price (most aggressive discount)
            target_price = valid_prices[0]
        else:
            # Fallback to margin-based prices
            target_margin = float(target_margin) if target_margin and not pd.isna(target_margin) else 0.10
            min_boundary = float(min_boundary) if min_boundary and not pd.isna(min_boundary) else 0.05
            
            if current_margin > target_margin and current_margin - target_margin > MIN_PRICE_REDUCTION_RATIO:
                target_price = wac / (1 - target_margin)
                source += '_target_margin'
            elif current_margin > min_boundary and current_margin - min_boundary > MIN_PRICE_REDUCTION_RATIO:
                target_price = wac / (1 - min_boundary)
                source += '_min_boundary'
            elif current_margin > target_margin / 2 and current_margin - target_margin / 2 > MIN_PRICE_REDUCTION_RATIO:
                target_price = wac / (1 - (target_margin / 2))
                source += '_half_target'
        
        return target_price, source
    
    # =========================================================================
    # CASE 2: Normal Products - Based on UTH Status
    # =========================================================================
    
    # Filter prices below current
    below_current = prices_arr[prices_arr < current_price]
    if len(below_current) == 0:
        return 0.0, 'no_lower_prices'
    
    # Calculate price changes
    diffs = (below_current - current_price) / current_price
    
    # -------------------------------------------------------------------------
    # UTH Status: On Track - Use old discount if available
    # -------------------------------------------------------------------------
    if uth_status in ['on_track', 'On Track']:
        if old_discount_price and old_discount_price > 0 and old_discount_price < current_price:
            # Validate old discount price is still acceptable
            if old_discount_price >= min_acceptable_price:
                return old_discount_price, 'on_track_keep_old'
        
        # No old discount - find moderate reduction (middle of valid range)
        valid_mask = (below_current >= min_acceptable_price) & (diffs >= -MAX_PRICE_REDUCTION_RATIO) & (diffs <= -MIN_PRICE_REDUCTION_RATIO)
        valid_prices = below_current[valid_mask]
        
        if len(valid_prices) > 0:
            # Take middle price
            target_price = valid_prices[len(valid_prices) // 2]
            source = 'on_track_moderate'
        
        return target_price, source
    
    # -------------------------------------------------------------------------
    # UTH Status: Dropping - More Aggressive (lower price than old discount)
    # -------------------------------------------------------------------------
    if uth_status in ['dropping', 'Dropping', 'critical', 'Critical', 'zero_demand', 'Zero Demand']:
        # Valid range: between -0.25% and -5% from current
        valid_mask = (below_current >= min_acceptable_price) & (diffs >= -MAX_PRICE_REDUCTION_RATIO) & (diffs <= -MIN_PRICE_REDUCTION_RATIO)
        valid_prices = below_current[valid_mask]
        
        if len(valid_prices) > 0:
            if old_discount_price and old_discount_price > 0:
                # Go LOWER than old discount (more aggressive)
                even_lower = valid_prices[valid_prices < old_discount_price]
                if len(even_lower) > 0:
                    target_price = even_lower[-1]  # Highest of the lower prices
                    source = 'dropping_below_old'
                else:
                    # No price below old discount, use lowest valid
                    target_price = valid_prices[0]
                    source = 'dropping_lowest'
            else:
                # No old discount - take lowest valid price (most aggressive)
                target_price = valid_prices[0]
                source = 'dropping_aggressive'
        
        return target_price, source
    
    # -------------------------------------------------------------------------
    # UTH Status: Growing - Less Aggressive (higher price than old discount)
    # -------------------------------------------------------------------------
    if uth_status in ['growing', 'Growing']:
        valid_mask = (below_current >= min_acceptable_price) & (diffs >= -MAX_PRICE_REDUCTION_RATIO) & (diffs <= -MIN_PRICE_REDUCTION_RATIO)
        valid_prices = below_current[valid_mask]
        
        if len(valid_prices) > 0:
            if old_discount_price and old_discount_price > 0:
                # Go HIGHER than old discount (less aggressive)
                higher_prices = valid_prices[valid_prices > old_discount_price]
                if len(higher_prices) > 0:
                    target_price = higher_prices[0]  # Lowest of the higher prices
                    source = 'growing_above_old'
                else:
                    # No prices higher than old discount - keep old discount
                    target_price = old_discount_price
                    source = 'growing_keep_old'
            else:
                # No old discount - take highest valid price (least aggressive)
                target_price = valid_prices[-1]
                source = 'growing_conservative'
        
        return target_price, source
    
    # -------------------------------------------------------------------------
    # Default: Find best price in valid range
    # -------------------------------------------------------------------------
    valid_mask = (below_current >= min_acceptable_price) & (diffs >= -MAX_PRICE_REDUCTION_RATIO) & (diffs <= -MIN_PRICE_REDUCTION_RATIO)
    valid_prices = below_current[valid_mask]
    
    if len(valid_prices) > 0:
        target_price = valid_prices[-1]  # Take highest (most conservative)
        source = 'default_valid'
    
    return target_price, source


def calculate_discount_for_row(row: pd.Series) -> pd.Series:
    """
    Process a single row to calculate target price and discount percentage.
    
    Args:
        row: Series with all pricing data
    
    Returns:
        Series with 'target_price', 'discount_percentage', 'discount_source'
    """
    # Use new_price if available, otherwise current_price
    new_price = row.get('new_price')
    if new_price and not pd.isna(new_price) and float(new_price) > 0:
        current_price = float(new_price)
    else:
        current_price = float(row.get('current_price', 0) or 0)
    
    wac = float(row.get('wac_p', 0) or 0)
    
    if current_price <= 0 or wac <= 0:
        return pd.Series({
            'target_price': 0.0,
            'discount_percentage': 0.0,
            'discount_source': 'invalid_data'
        })
    
    # Get parameters
    target_margin = float(row.get('target_margin', 0.10) or 0.10)
    min_boundary = float(row.get('min_boundary', 0.05) or 0.05)
    doh = float(row.get('doh', 0) or 0)
    # For SKU discounts, only use the zero_demand flag (ignore uth_qty)
    is_zero_demand = bool(row.get('zero_demand', False))
    uth_status = str(row.get('uth_status', 'dropping'))
    
    # Calculate old discount price if there was an active discount
    active_disc_pct = float(row.get('active_sku_disc_pct', 0) or 0)
    if active_disc_pct > 0:
        old_discount_price = current_price * (1 - active_disc_pct / 100)
    else:
        old_discount_price = None
    
    # Build candidate prices
    candidate_prices = build_candidate_prices(row)
    
    if len(candidate_prices) == 0:
        return pd.Series({
            'target_price': 0.0,
            'discount_percentage': 0.0,
            'discount_source': 'no_candidates'
        })
    
    # Select target price
    target_price, source = select_target_price(
        candidate_prices=candidate_prices,
        current_price=current_price,
        wac=wac,
        target_margin=target_margin,
        min_boundary=min_boundary,
        is_zero_demand=is_zero_demand,
        doh=doh,
        uth_status=uth_status,
        old_discount_price=old_discount_price
    )
    
    # Calculate discount percentage
    if target_price > 0 and target_price < current_price:
        discount = abs((target_price - current_price) / current_price)
        
        # Round to nearest 0.1% (same as HH-V3)
        discount = discount * 100000
        discount = ((discount // 10) + 1) / 10000
        
        # Cap at max discount
        discount = min(discount, MAX_DISCOUNT_PERCENT / 100)
        
        # Convert to percentage
        discount_pct = discount * 100
        
        # Skip if below minimum
        if discount_pct < MIN_DISCOUNT_PERCENT:
            discount_pct = 0.0
            source = 'below_min_threshold'
    else:
        discount_pct = 0.0
        if source == '':
            source = 'no_reduction_needed'
    
    return pd.Series({
        'target_price': target_price,
        'discount_percentage': round(discount_pct, 2),
        'discount_source': source
    })


def calculate_discounts_batch(df_skus: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate discounts for all SKUs in batch.
    
    Args:
        df_skus: DataFrame with SKUs needing discounts
    
    Returns:
        DataFrame with 'target_price', 'discount_percentage', 'discount_source' columns added
    """
    print(f"Calculating discounts for {len(df_skus)} SKUs...")
    
    # Enable progress bar
    tqdm.pandas(desc="Calculating discounts")
    
    # Apply discount calculation to each row
    result_cols = df_skus.progress_apply(calculate_discount_for_row, axis=1)
    
    # Combine with original data
    df_result = df_skus.copy()
    df_result['target_price'] = result_cols['target_price']
    df_result['discount_percentage'] = result_cols['discount_percentage']
    df_result['discount_source'] = result_cols['discount_source']
    
    # Summary statistics
    valid_discounts = df_result[df_result['discount_percentage'] > 0]
    print(f"\n  ✓ Discounts calculated:")
    print(f"    - Valid discounts: {len(valid_discounts)}")
    print(f"    - Avg discount: {valid_discounts['discount_percentage'].mean():.2f}%" if len(valid_discounts) > 0 else "    - Avg discount: N/A")
    print(f"    - Discount sources: {df_result['discount_source'].value_counts().to_dict()}")
    
    return df_result


print("Function 3: Price selection & discount calculation defined ✓")
print("  - margin_to_price()")
print("  - build_candidate_prices()")
print("  - select_target_price()")
print("  - calculate_discount_for_row()")
print("  - calculate_discounts_batch()")


In [ ]:
# =============================================================================
# FUNCTION 4: STRUCTURE DATAFRAME
# =============================================================================
import shutil
from pathlib import Path

# Output folder for SKU discount sheets
SKU_DISCOUNT_OUTPUT_FOLDER = '../output/sku_discount_sheets'
SKU_DISCOUNT_TEMP_FOLDER = '../output/sku_discount_temp'

# Max retailers per API call
MAX_RETAILERS_PER_CHUNK = 100

# Max rows per Excel file
MAX_ROWS_PER_FILE = 1000


def clear_output_folder(folder_path: str, temp_folder: str) -> None:
    """
    Clear all files from the output folder by moving them to temp.
    
    Args:
        folder_path: Path to the folder to clear
        temp_folder: Path to move files to before deletion
    """
    # Create folders if they don't exist
    os.makedirs(folder_path, exist_ok=True)
    os.makedirs(temp_folder, exist_ok=True)
    
    source = Path(folder_path)
    
    moved_count = 0
    error_count = 0
    
    for file in source.iterdir():
        if file.is_file():
            try:
                # Delete file
                file.unlink()
                moved_count += 1
            except Exception as e:
                print(f"  ✗ Error deleting {file.name}: {e}")
                error_count += 1
    
    if moved_count > 0:
        print(f"  Cleared {moved_count} files from output folder")


def structure_sku_discount_dataframe(df_skus: pd.DataFrame) -> pd.DataFrame:
    """
    Structure the DataFrame in the format expected by the SKU discount API/upload.
    
    Transforms the internal data format into the API-expected format with proper 
    column names, data types, grouping by discount, and chunking for upload.
    
    Process:
    1. Deduplicate SKUs (keep lowest discount per product-warehouse-retailer)
    2. Merge with packing units
    3. Create HH_data format: [product_id, packing_unit_id, discount]
    4. Set start/end times (start = now + 10 mins, end = start + 12 hours)
    5. Group by retailer to create discount lists
    6. Group by discounts to aggregate retailers
    7. Chunk retailer lists (max 100 per chunk)
    8. Add required columns (offer names)
    
    Args:
        df_skus: DataFrame with discount calculations and retailer targeting
    
    Returns:
        DataFrame ready for API upload with columns:
            - Retailers List
            - Start Date/Time
            - End Date/Time
            - Discounts
            - Arabic Offer Name
            - French Offer Name
            - English Offer Name
            - Swahili Offer Name
            - Rwandan Offer Name
    """
    print(f"Structuring {len(df_skus)} SKU discount records for API...")
    
    if df_skus.empty:
        print("  No records to structure")
        return pd.DataFrame()
    
    df = df_skus.copy()
    
    # =========================================================================
    # Step 1: Deduplicate - keep lowest discount per product-warehouse-retailer
    # =========================================================================
    print("  Step 1: Deduplicating...")
    df = df.sort_values('discount_percentage').drop_duplicates(
        subset=['product_id', 'warehouse_id', 'retailer_id']
    )
    print(f"    Records after deduplication: {len(df)}")
    
    # =========================================================================
    # Step 2: Merge with packing units
    # =========================================================================
    print("  Step 2: Merging with packing units...")
    try:
        pus = get_packing_units()
        df = df.merge(pus, on='product_id', how='left')
        df = df.drop_duplicates()
        print(f"    Records after PU merge: {len(df)}")
    except Exception as e:
        print(f"    ⚠ Failed to get packing units: {e}")
        # Use product_id as packing_unit_id fallback
        df['packing_unit_id'] = df['product_id']
    
    # =========================================================================
    # Step 3: Create HH_data format
    # =========================================================================
    print("  Step 3: Creating HH_data format...")
    df['HH_data'] = '[' + df['product_id'].astype(str) + ',' + \
                    df['packing_unit_id'].astype(str) + ',' + \
                    df['discount_percentage'].astype(str) + ']'
    
    # =========================================================================
    # Step 4: Set start and end times (Cairo timezone)
    # =========================================================================
    print("  Step 4: Setting start/end times...")
    start_time = datetime.now(CAIRO_TZ) + timedelta(minutes=10)
    end_time = start_time + timedelta(hours=12)
    
    # Format for API: "YYYY-MM-DD HH:MM" (without seconds)
    start_date = start_time.strftime('%Y-%m-%d %H:%M')
    end_date = end_time.strftime('%Y-%m-%d %H:%M')
    print(f"    Start: {start_date}")
    print(f"    End: {end_date}")
    
    # =========================================================================
    # Step 5: Group by retailer_id to create discount lists
    # =========================================================================
    print("  Step 5: Grouping by retailer...")
    output_df = df.groupby('retailer_id')['HH_data'].apply(list).reset_index()
    output_df['Discounts'] = output_df['HH_data'].astype(str).str.replace("'", '').str.replace(' ', '')
    print(f"    Unique retailers: {len(output_df)}")
    
    # =========================================================================
    # Step 6: Group by Discounts to aggregate retailers
    # =========================================================================
    print("  Step 6: Grouping by discount combinations...")
    output_df = output_df.groupby('Discounts')['retailer_id'].agg(list).reset_index()
    print(f"    Unique discount combinations: {len(output_df)}")
    
    # Add required columns
    output_df['Arabic Offer Name'] = 'خصومات حصرية'
    output_df['Start Date/Time'] = start_date
    output_df['End Date/Time'] = end_date
    output_df = output_df[['retailer_id', 'Start Date/Time', 'End Date/Time', 'Discounts', 'Arabic Offer Name']]
    output_df['French Offer Name'] = np.nan
    output_df['English Offer Name'] = np.nan
    
    # =========================================================================
    # Step 7: Chunk retailer lists (max 100 per chunk)
    # =========================================================================
    print(f"  Step 7: Chunking retailer lists (max {MAX_RETAILERS_PER_CHUNK} per chunk)...")
    data = []
    for i, row in output_df.iterrows():
        start_date_val = row['Start Date/Time']
        end_date_val = row['End Date/Time']
        retailers = row['retailer_id']
        discount = row['Discounts']
        name = row['Arabic Offer Name']
        name_f = row['French Offer Name']
        name_e = row['English Offer Name']
        
        length = len(retailers)
        if length > MAX_RETAILERS_PER_CHUNK:
            # Split into chunks of MAX_RETAILERS_PER_CHUNK
            iters = length // MAX_RETAILERS_PER_CHUNK
            for j in range(0, iters + 1):
                start_idx = j * MAX_RETAILERS_PER_CHUNK
                end_idx = (j + 1) * MAX_RETAILERS_PER_CHUNK
                rets = retailers[start_idx:end_idx]
                if len(rets) > 0:
                    data.append({
                        'Discounts': discount,
                        'retailer_id': rets,
                        'Start Date/Time': start_date_val,
                        'End Date/Time': end_date_val,
                        'Arabic Offer Name': name,
                        'French Offer Name': name_f,
                        'English Offer Name': name_e
                    })
        else:
            data.append({
                'Discounts': discount,
                'retailer_id': retailers,
                'Start Date/Time': start_date_val,
                'End Date/Time': end_date_val,
                'Arabic Offer Name': name,
                'French Offer Name': name_f,
                'English Offer Name': name_e
            })
    
    dfx = pd.DataFrame(data)
    print(f"    Total chunks: {len(dfx)}")
    
    # =========================================================================
    # Step 8: Add extra columns and rename
    # =========================================================================
    print("  Step 8: Finalizing columns...")
    dfx['English Offer Name'] = 'Special Discounts'
    dfx['Swahili Offer Name'] = ''
    dfx['Rwandan Offer Name'] = ''
    
    # Rename and reorder columns
    dfx.rename(columns={'retailer_id': 'Retailers List'}, inplace=True)
    dfx = dfx[[
        'Retailers List', 'Start Date/Time', 'End Date/Time', 'Discounts',
        'Arabic Offer Name', 'French Offer Name', 'English Offer Name',
        'Swahili Offer Name', 'Rwandan Offer Name'
    ]]
    
    print(f"  ✓ Structured {len(dfx)} records for upload")
    
    return dfx


def save_sku_discount_files(df_structured: pd.DataFrame) -> list:
    """
    Save structured SKU discount data to Excel files.
    
    Clears the output folder first, then saves files with max 1000 rows each.
    
    Args:
        df_structured: Structured DataFrame from structure_sku_discount_dataframe
    
    Returns:
        List of saved file paths
    """
    if df_structured.empty:
        print("No data to save")
        return []
    
    print(f"\nSaving SKU discount files...")
    
    # Clear output folder
    print("  Clearing output folder...")
    clear_output_folder(SKU_DISCOUNT_OUTPUT_FOLDER, SKU_DISCOUNT_TEMP_FOLDER)
    
    # Create output folder if needed
    os.makedirs(SKU_DISCOUNT_OUTPUT_FOLDER, exist_ok=True)
    
    # Reset index
    final = df_structured.reset_index(drop=True)
    
    # Calculate file ranges
    mino = final.index.min()
    maxo = final.index.max()
    ran = [i for i in range(mino, maxo + 1, MAX_ROWS_PER_FILE)]
    
    saved_files = []
    
    print(f"  Saving {len(ran)} files (max {MAX_ROWS_PER_FILE} rows each)...")
    
    for i in tqdm(range(len(ran)), desc="Saving files"):
        if i + 1 == len(ran):
            val1 = ran[i]
            val2 = maxo
        else:
            val1 = ran[i]
            val2 = ran[i + 1] - 1
        
        x = final.loc[val1:val2, :]
        
        # Generate filename with date
        filename = f'sku_discount_{str(datetime.now(CAIRO_TZ).date())}_NO._{i}.xlsx'
        filepath = os.path.join(SKU_DISCOUNT_OUTPUT_FOLDER, filename)
        
        x.to_excel(filepath, index=False)
        saved_files.append(filepath)
    
    print(f"  ✓ Saved {len(saved_files)} files to {SKU_DISCOUNT_OUTPUT_FOLDER}")
    
    return saved_files


print("Function 4: structure_sku_discount_dataframe() defined ✓")
print("  - clear_output_folder()")
print("  - structure_sku_discount_dataframe()")
print("  - save_sku_discount_files()")


In [ ]:
# =============================================================================
# FUNCTION 5: PUSH SKU DISCOUNT (via S3 Upload)
# =============================================================================
# Reference: HH-V3.ipynb upload flow
# Flow: Save Excel → Get pre-signed URL → Upload to S3 → Validate → Proceed

def _get_presigned_url() -> dict:
    """
    Get a pre-signed S3 URL for uploading SKU discount Excel file.
    
    Returns:
        dict with 'key' and 'preSignedUrl'
    """
    token = _get_api_token()
    url = "https://api.maxab.info/commerce/api/admins/v1/bulk-upload/presigned-url?type=SKU_DISCOUNTS"
    
    headers = {
        'Authorization': f'bearer {token}'
    }
    
    response = requests.get(url, headers=headers)
    return response.json()


def _upload_file_to_s3(file_path: str, presigned_url: str) -> requests.Response:
    """
    Upload Excel file to S3 using pre-signed URL.
    
    Args:
        file_path: Path to Excel file
        presigned_url: Pre-signed S3 URL
    
    Returns:
        Response object
    """
    headers = {'Content-Type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}
    
    with open(file_path, 'rb') as f:
        response = requests.put(presigned_url, data=f, headers=headers)
    
    return response


def _validate_sku_discount(key: str) -> requests.Response:
    """
    Validate uploaded SKU discount file.
    
    Args:
        key: S3 key returned from presigned URL
    
    Returns:
        Response object
    """
    token = _get_api_token()
    url = 'https://api.maxab.info/commerce/api/admins/v1/bulk-upload/sheets/validate'
    
    headers = {
        'Authorization': f'bearer {token}',
        'content-type': 'application/json'
    }
    
    payload = {"fileName": key, "sheetType": "SKU_DISCOUNTS"}
    response = requests.post(url, headers=headers, json=payload)
    
    return response


def _proceed_sku_discount(key: str) -> requests.Response:
    """
    Proceed with processing the validated SKU discount file.
    
    Args:
        key: S3 key returned from presigned URL
    
    Returns:
        Response object
    """
    token = _get_api_token()
    url = f'https://api.maxab.info/commerce/api/admins/v1/bulk-upload/sheets/proceed/{key}?uploadType=SKU_DISCOUNTS'
    
    headers = {
        'Authorization': f'bearer {token}',
        'content-type': 'application/json'
    }
    
    response = requests.post(url, headers=headers)
    return response


def _upload_single_file(file_path: str) -> dict:
    """
    Upload a single SKU discount Excel file through the full pipeline.
    
    Flow:
    1. Get pre-signed URL
    2. Upload file to S3
    3. Validate file
    4. Proceed with processing
    
    Args:
        file_path: Path to Excel file
    
    Returns:
        dict with upload results
    """
    result = {
        'file': file_path,
        'timestamp': datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S'),
        'status': 'PENDING',
        'steps': {}
    }
    
    try:
        # Step 1: Get pre-signed URL
        pre_data = _get_presigned_url()
        key = pre_data['key']
        presigned_url = pre_data['preSignedUrl']
        result['steps']['presigned_url'] = 'Success'
        result['key'] = key
        
    except Exception as e:
        result['steps']['presigned_url'] = f'Failed: {e}'
        result['status'] = 'FAILED'
        return result
    
    try:
        # Step 2: Upload file to S3
        upload_response = _upload_file_to_s3(file_path, presigned_url)
        
        if upload_response.status_code in [200, 201, 204]:
            result['steps']['upload'] = 'Success'
        else:
            result['steps']['upload'] = f'Failed: {upload_response.status_code}'
            result['status'] = 'FAILED'
            return result
            
    except Exception as e:
        result['steps']['upload'] = f'Failed: {e}'
        result['status'] = 'FAILED'
        return result
    
    try:
        # Step 3: Validate file
        validation_response = _validate_sku_discount(key)
        
        if validation_response.ok:
            result['steps']['validation'] = 'Success'
        else:
            result['steps']['validation'] = f'Failed: {validation_response.status_code}'
            result['status'] = 'FAILED'
            return result
            
    except Exception as e:
        result['steps']['validation'] = f'Failed: {e}'
        result['status'] = 'FAILED'
        return result
    
    try:
        # Step 4: Proceed with processing
        proceed_response = _proceed_sku_discount(key)
        
        if proceed_response.ok:
            result['steps']['proceed'] = 'Success'
            result['status'] = 'SUCCESS'
        else:
            result['steps']['proceed'] = f'Failed: {proceed_response.status_code}'
            result['status'] = 'FAILED'
            
    except Exception as e:
        result['steps']['proceed'] = f'Failed: {e}'
        result['status'] = 'FAILED'
    
    return result


def push_sku_discount(df_api: pd.DataFrame, mode: str = 'testing') -> dict:
    """
    Push SKU discounts - saves files and optionally uploads via S3.
    
    Flow (live mode):
    1. Save Excel files to output folder
    2. For each file:
       a. Get pre-signed S3 URL
       b. Upload file to S3
       c. Validate file
       d. Proceed with processing
    
    Args:
        df_api: DataFrame structured for API (from structure_sku_discount_dataframe)
        mode: 'testing' (save files only) or 'live' (save files and upload via S3)
    
    Returns:
        dict with:
            - success: bool
            - created_count: int
            - failed_count: int
            - saved_files: list of file paths
            - upload_results: list of upload results per file
            - errors: list
    """
    print(f"\n{'🧪' if mode == 'testing' else '🚀'} MODE: {mode.upper()}")
    
    result = {
        "success": True,
        "created_count": 0,
        "failed_count": 0,
        "saved_files": [],
        "upload_results": [],
        "errors": []
    }
    
    if df_api.empty:
        print("No SKU discounts to push")
        return result
    
    print(f"Processing {len(df_api)} SKU discount records...")
    
    # Step 1: Save files to output folder
    print("\n  Step 1: Saving files to output folder...")
    saved_files = save_sku_discount_files(df_api)
    result['saved_files'] = saved_files
    
    if mode == 'testing':
        print(f"\n  🧪 [TESTING] Files saved but S3 upload skipped")
        print(f"  Files location: {SKU_DISCOUNT_OUTPUT_FOLDER}")
        print(f"  Files saved: {len(saved_files)}")
        result['created_count'] = len(df_api)
        return result
    
    # Step 2: Upload each file via S3 (live mode)
    print(f"\n  Step 2: Uploading {len(saved_files)} files via S3...")
    
    success_count = 0
    failed_count = 0
    
    for file_path in tqdm(saved_files, desc="Uploading files"):
        print(f"\n    Processing: {os.path.basename(file_path)}")
        
        upload_result = _upload_single_file(file_path)
        result['upload_results'].append(upload_result)
        
        if upload_result['status'] == 'SUCCESS':
            success_count += 1
            print(f"      ✓ Success")
        else:
            failed_count += 1
            print(f"      ✗ Failed: {upload_result['steps']}")
            result['errors'].append({
                'file': file_path,
                'steps': upload_result['steps']
            })
    
    result['created_count'] = success_count
    result['failed_count'] = failed_count
    result['success'] = failed_count == 0
    
    # Summary
    print(f"\n  {'='*50}")
    print(f"  UPLOAD SUMMARY")
    print(f"  {'='*50}")
    print(f"  Total files: {len(saved_files)}")
    print(f"  ✓ Successful: {success_count}")
    print(f"  ✗ Failed: {failed_count}")
    
    if failed_count > 0:
        print(f"\n  Failed files:")
        for err in result['errors'][:5]:  # Show first 5
            print(f"    - {os.path.basename(err['file'])}")
    
    return result


print("Function 5: push_sku_discount() defined ✓")
print("  - _get_presigned_url()")
print("  - _upload_file_to_s3()")
print("  - _validate_sku_discount()")
print("  - _proceed_sku_discount()")
print("  - _upload_single_file()")
print("  - push_sku_discount()")


In [ ]:
# =============================================================================
# MAIN ORCHESTRATION FUNCTION
# =============================================================================

def process_sku_discounts(
    df_module3_output: pd.DataFrame,
    mode: str = 'testing'
) -> dict:
    """
    Main entry point: Process SKU discounts from Module 3 output.
    
    Note: Market data should already be refreshed in Module 3 before calling this.
    
    Orchestrates the full workflow:
    1. Deactivate existing SKU discounts
    2. Filter to SKUs needing discounts (activate_sku_discount = True)
    3. Apply exclusions (categories, brands)
    4. Select target retailers
    5. Calculate discount percentages using HH-V3 logic
    6. Structure DataFrame for API
    7. Push to API
    
    Args:
        df_module3_output: DataFrame from Module 3 with columns:
            - product_id, warehouse_id, sku, cohort_id
            - activate_sku_discount (bool)
            - current_price, wac_p, doh, zero_demand
            - uth_status, active_sku_disc_pct
            - Raw market prices, market margins, margin boundaries, margin tiers
        mode: 'testing' or 'live'
    
    Returns:
        dict with summary of all operations
    """
    print("\n" + "="*70)
    print("SKU DISCOUNT HANDLER")
    print("="*70)
    print(f"Mode: {mode}")
    print(f"Input records: {len(df_module3_output)}")
    
    result = {
        'timestamp': CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S'),
        'mode': mode,
        'total_input': len(df_module3_output),
        'excluded_category': 0,
        'excluded_brand': 0,
        'to_activate': 0,
        'deactivated': 0,
        'created': 0,
        'failed': 0
    }
    
    df = df_module3_output.copy()
    
    # Step 1: Deactivate existing discounts
    print("\n" + "-"*50)
    print("STEP 1: Deactivating existing SKU discounts")
    print("-"*50)
    deactivate_result = deactivate_active_sku_discounts(mode=mode)
    result['deactivated'] = deactivate_result['deactivated_count']
    
    # Step 2: Filter to SKUs needing discounts
    print("\n" + "-"*50)
    print("STEP 2: Filtering SKUs for discount")
    print("-"*50)
    df_to_activate = df[
        df['activate_sku_discount'] == True
    ].copy()
    initial_count = len(df_to_activate)
    print(f"SKUs flagged for discount: {initial_count}")
    
    # Step 2b: Apply exclusions
    print("\n  Applying exclusions...")
    
    # Exclude categories
    if 'cat' in df_to_activate.columns and len(EXCLUDED_CATEGORIES) > 0:
        before = len(df_to_activate)
        df_to_activate = df_to_activate[~df_to_activate['cat'].isin(EXCLUDED_CATEGORIES)]
        excluded_cat = before - len(df_to_activate)
        result['excluded_category'] = excluded_cat
        print(f"    - Excluded by category: {excluded_cat}")
    
    # Exclude brands
    if 'brand' in df_to_activate.columns and len(EXCLUDED_BRANDS) > 0:
        before = len(df_to_activate)
        df_to_activate = df_to_activate[~df_to_activate['brand'].isin(EXCLUDED_BRANDS)]
        excluded_brand = before - len(df_to_activate)
        result['excluded_brand'] = excluded_brand
        print(f"    - Excluded by brand: {excluded_brand}")
    
    result['to_activate'] = len(df_to_activate)
    print(f"\n  Final SKUs to activate: {len(df_to_activate)}")
    
    if df_to_activate.empty:
        print("No SKUs need discounts. Done.")
        return result
    
    # Step 3: Calculate discounts (BEFORE retailer selection)
    # We need to know which SKUs have valid discounts before selecting retailers
    print("\n" + "-"*50)
    print("STEP 3: Calculating discount percentages")
    print("-"*50)
    df_with_discounts = calculate_discounts_batch(df_to_activate)
    
    # Filter to SKUs with valid discounts
    df_valid_discounts = df_with_discounts[df_with_discounts['discount_percentage'] > 0].copy()
    result['valid_discounts'] = len(df_valid_discounts)
    print(f"\n  SKUs with valid discounts (>0%): {len(df_valid_discounts)}")
    
    if df_valid_discounts.empty:
        print("No SKUs with valid discounts. Done.")
        return result
    
    # Step 4: Select target retailers (only for valid discounts)
    print("\n" + "-"*50)
    print("STEP 4: Selecting target retailers")
    print("-"*50)
    df_with_retailers = select_target_retailers(df_valid_discounts)
    
    if len(df_with_retailers) == 0 or df_with_retailers['retailer_id'].isna().all():
        print("No retailers found for any SKU. Done.")
        return result
    
    result['retailer_product_combinations'] = len(df_with_retailers)
    
    # Step 5: Structure DataFrame
    print("\n" + "-"*50)
    print("STEP 5: Structuring data for API")
    print("-"*50)
    df_api = structure_sku_discount_dataframe(df_with_retailers)
    
    # Step 6: Push to API
    print("\n" + "-"*50)
    print("STEP 6: Pushing to API")
    print("-"*50)
    push_result = push_sku_discount(df_api, mode=mode)
    result['created'] = push_result['created_count']
    result['failed'] = push_result['failed_count']
    
    # Summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    print(f"Mode: {result['mode']}")
    print(f"Total input: {result['total_input']}")
    print(f"Discounts deactivated: {result['deactivated']}")
    print(f"SKUs to activate: {result['to_activate']}")
    print(f"SKUs with valid discounts: {result.get('valid_discounts', 0)}")
    print(f"Retailer-product combinations: {result.get('retailer_product_combinations', 0)}")
    print(f"Records created/uploaded: {result['created']}")
    print(f"Records failed: {result['failed']}")
    if 'saved_files' in push_result:
        print(f"Files saved: {len(push_result['saved_files'])}")
        print(f"Output folder: {SKU_DISCOUNT_OUTPUT_FOLDER}")
    
    return result


print("Main function: process_sku_discounts() defined ✓")


In [ ]:
# =============================================================================
# USAGE EXAMPLE (Called from Module 3)
# =============================================================================
# 
# This module is designed to be called from Module 3 after the periodic
# action engine has determined which SKUs need discounts.
#
# Example usage:
#
# %run sku_discount_handler.ipynb
#
# # df_output already contains market margins and margin tiers from Module 3
# # Required columns:
# #   - product_id, warehouse_id, sku, cohort_id
# #   - activate_sku_discount (bool)
# #   - current_price, new_price, wac_p
# #   - doh, uth_qty, uth_status, active_sku_disc_pct
# #   - target_margin, min_boundary
# #   - below_market, market_min, market_25, market_50, market_75, market_max, above_market
# #   - margin_tier_below, margin_tier_1, ..., margin_tier_above_2
#
# result = process_sku_discounts(df_output, mode='testing')
#
# print(f"Created {result['created']} SKU discounts")

print("\n" + "="*60)
print("SKU DISCOUNT HANDLER MODULE READY")
print("="*60)
print("\nRequired input columns from Module 3:")
print("  - product_id, warehouse_id, sku, cohort_id, brand, cat")
print("  - activate_sku_discount (bool)")
print("  - current_price, new_price, wac_p")
print("  - doh, uth_qty, uth_status, active_sku_disc_pct")
print("  - target_margin, min_boundary")
print("\nRequired market margin columns (prices derived via wac/(1-margin)):")
for col in MARKET_MARGIN_COLS:
    print(f"  - {col}")
print("\nRequired margin tier columns:")
for col in MARGIN_TIER_COLS:
    print(f"  - {col}")
print("\nRetailer Selection: Queries 4 sources, applies exclusions, removes QD overlap")
print("\nUsage: result = process_sku_discounts(df_output, mode='testing')")
